In [1]:
from scipy.stats import nbinom, binom, poisson
from statsmodels.discrete.count_model import ZeroInflatedNegativeBinomialP
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import math
import seaborn as sns
from datetime import datetime, timedelta
from scipy.stats import pearsonr
from scipy.stats import kendalltau
from scipy.stats import spearmanr
import os
from scipy.optimize import minimize, Bounds, LinearConstraint, NonlinearConstraint
import inspect

### Relevante Parameter:

array_lambda = np.exp(np.log(0.63*EM_monat) + alpha_o[state] + beta_o[0] * np.log(LO_monat))

Die Öffungszahlen sind mit einer Poissonverteilung modelliert und die Werte folgender Tabellen haben Einfluss:

1. EM_monat
2. LO_monat

folgende Parameter haben einen Einfluss:
1. alpha_o (1 x anzahl_states array, also state abhängig)
2. beta_o (1 x 1 array, state unabhängig)

In [2]:
# EM_monat aus der HY Zahlen der versendeten Mails ziehen
path = r'C:/Users/felix.oechslein/Desktop/Analysen Thalia, Mai/verarbeitete_daten'
filename = 'Pivot_HY_Privat_ohneApple_Langzeitkunde.csv'

df_HY = pd.read_csv(os.path.join(path, filename))

In [3]:
# LO monat
path = r'C:/Users/felix.oechslein/Desktop/Analysen Thalia, Mai/verarbeitete_daten'
filename = 'Pivot_LO_Privat_ohneApple_Langzeitkunde.csv'

df_LO = pd.read_csv(os.path.join(path, filename))

### Datenanpassung wie bei der Berechnung der optimalen Parameter

In [4]:
## Alle DataFrame Werte von Float zu int
df_HY = df_HY.astype(int)
df_LO = df_LO.astype(int)

### Sicherstellen dass alle df dieselben Kunden verwenden

In [5]:
# Entfernen der Zeilen aus df_LO, die nicht in df_HY enthalten sind
df_LO = df_LO[df_LO['iid'].isin(df_HY['iid'])]
df_HY = df_HY[df_HY['iid'].isin(df_LO['iid'])]

In [6]:
len(df_LO)

647

In [7]:
len(df_HY)

647

### Ausreißer bei den Kunden löschen (wie bei den Berechnungen)

In [8]:
kunden_pos = [384, 446, 502, 599, 421, 285, 407, 499, 348, 351, 364, 449, 528, 414]

In [9]:
# Zeilen löschen
df_HY = df_HY.drop(kunden_pos).reset_index(drop = True)
df_LO = df_LO.drop(kunden_pos).reset_index(drop = True)

### Spezifikation des Modells

In [10]:
anzahl_kunden = len(df_HY["iid"])
anzahl_kunden

633

In [11]:
anzahl_monate = len(df_HY.columns[df_HY.columns != 'iid'])
anzahl_monate

15

In [12]:
anzahl_states = 3

### 1. Herauslesen der Parameter aus dem array der gesamten geschätzten Parameter

In [13]:
estimted_params=[ -8.20399103e-01,  4.47364379e+01,  6.55459562e+01, -7.12664184e-01,
       -2.48373911e+02,  4.63946330e+02, -1.89923145e+00, -5.94099736e+01,
        7.59497882e+01,  5.18311697e-01, -2.61490366e-01,  8.93093831e-01,
        1.63703041e-01,  7.28623716e-03, -2.62212305e-01,  9.68519753e-02,
        8.66641236e-02,  1.82355786e-01,  2.36506775e-01, -8.98038162e-02,
       -4.25975327e-01,  4.99874934e-01, -1.43621259e-01,  8.11571266e-01,
        1.42871394e+00, -7.33210161e-01,  2.96600857e-01,  1.65401014e-01,
        1.01422490e-02, -3.14895811e-01, -2.02066317e-01, -5.13156402e-01,
        3.44435091e-01,  1.23401987e+00, -1.31647262e+00, -1.48696412e+00,
       -6.23244984e-01,  1.30427990e+00, -6.95618496e-01, -5.10706288e-01,
        4.85842039e-01, -5.54033892e-01,  8.96741073e-02,  9.01273778e-02,
       -3.67011581e-01, -2.31574553e-02, -6.96592721e-01,  1.16671348e+00,
       -1.26661223e+00,  1.23556052e+02, -2.55630320e+01, -4.60157041e+00,
        4.69496096e-01, -4.73224796e-01,  1.61440816e-01, -7.12740205e-01,
       -1.95849638e+01, -1.30792666e-01,  2.79970169e-01, -2.96108429e-01,
        1.53815838e+00,  1.00040668e-01, -1.41070531e-02, -4.57917675e-01,
        1.49032916e+00]

In [14]:
alpha_size = anzahl_states**2
gamma_size = anzahl_states**2 * 4

alpha = estimted_params[:alpha_size]
gamma = estimted_params[alpha_size:alpha_size+gamma_size]
alpha_o = estimted_params[alpha_size+gamma_size:alpha_size+gamma_size+anzahl_states]
beta_o = estimted_params[alpha_size+gamma_size+anzahl_states:alpha_size+gamma_size+anzahl_states+1]
delta_0 = estimted_params[alpha_size+gamma_size+anzahl_states+1:alpha_size+gamma_size+anzahl_states*2+1]
delta_1 = estimted_params[alpha_size+gamma_size+anzahl_states*2+1:alpha_size+gamma_size+anzahl_states*3+1]
alpha_p = estimted_params[alpha_size+gamma_size+anzahl_states*3+1:alpha_size+gamma_size+anzahl_states*4+1]
beta_p_1 = estimted_params[alpha_size+gamma_size+anzahl_states*4+1:alpha_size+gamma_size+anzahl_states*5+1]
beta_p_2 = estimted_params[alpha_size+gamma_size+anzahl_states*5+1:alpha_size+gamma_size+anzahl_states*6+1]
theta = estimted_params[alpha_size+gamma_size+anzahl_states*6+1:]

In [15]:
alpha_o

[-0.0231574553, -0.696592721, 1.16671348]

In [16]:
beta_o

[-1.26661223]

In [20]:
delta_1

[0.469496096, -0.473224796, 0.161440816]

In [21]:
alpha_p

[-0.712740205, -19.5849638, -0.130792666]

In [22]:
beta_p_1

[0.279970169, -0.296108429, 1.53815838]

In [23]:
beta_p_2

[0.100040668, -0.0141070531, -0.457917675]

### Erstellen der erwarteten Öffungen in den jeweiligen Monaten

In [24]:
erwartete_oeffungen = np.zeros((anzahl_kunden, anzahl_states, anzahl_monate))

In [25]:
for kunde in range(0, anzahl_kunden):    
    for state in range(0, anzahl_states):
        for monat in range(1, anzahl_monate +1):
            erwartete_oeffungen[kunde, state-1, monat-1] =\
                    np.exp(np.log(0.63)*df_HY.iloc[kunde][str(monat)] + alpha_o[state] +\
                           beta_o[0] * np.log(df_LO.iloc[kunde][str(monat)]))

In [26]:
np.sum(erwartete_oeffungen[:,0,:])/erwartete_oeffungen[:,0,:].size

0.16904649031151056

In [27]:
np.sum(erwartete_oeffungen[:,1,:])/erwartete_oeffungen[:,1,:].size

1.0895063347076048

In [28]:
np.sum(erwartete_oeffungen[:,2,:])/erwartete_oeffungen[:,1,:].size

0.3314937757373038